In [1]:
import pandas as pd
import selenium
import requests
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from bs4 import BeautifulSoup

In [36]:
#Intentar leer la tabla desde el html directamente
tablas = pd.read_html('https://hectoremmanuel88.wordpress.com/2017/09/01/top-100-canciones-en-espanol-decada-de-los-2000s-2000-2009/')
print(len(tablas))

3


In [37]:
songsDF = pd.DataFrame(tablas[0])
songsDF.head()

,0,1,2
0,100,La Ley y Ely Guerra . El duelo,2005
1,99,Natalia Lafourcade . Busca un problema,2002
2,98,Beto Cuevas . Vuelvo,2008
3,97,Entre Ríos . Si fuimos,2000
4,96,Litzy . No hay palabras,2000


In [38]:
songsDF = songsDF.sort_values(by = 0)
songsDF = songsDF.drop(columns = 0)
songsDF.columns = ['Song', 'Year']

In [3]:
# URL de la página
url = "https://www.letras.com/playlists/1006000/"

# Realizar la solicitud GET a la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Crear un objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar la lista de canciones
    song_list = soup.find('ul', class_="songList-table-content")

    # Lista para almacenar las canciones y artistas
    songs = []

    # Buscar todas las filas de canciones
    for song in song_list.find_all('li', class_="songList-table-row"):
        # Extraer el nombre de la canción y el artista
        song_name = song.find('div', class_="songList-table-songName").text.strip()
        artist_name = song.find('div', class_="songList-table-songArtist").text.strip()

        # Almacenar los datos en una lista
        songs.append({
            'Song': song_name,
            'Artist': artist_name
        })

    # Imprimir las canciones extraídas
    for song in songs:
        print(f"{song['Artist']} - {song['Song']}")

else:
    print(f"Error al obtener la página: {response.status_code}")

Britney Spears - Oops!... I Did It Again
U2 - Beautiful Day
Red Hot Chili Peppers - Californication
Coldplay - Yellow
John Mayer - Your Body Is a Wonderland
The White Stripes - Seven Nation Army
Eminem - Lose Yourself
Beyoncé - Crazy In Love (feat. Jay-Z)
Linkin Park - In The End
Avril Lavigne - Complicated
Norah Jones - Don't Know Why
Jota Quest - Amor Maior
3 Doors Down - Here Without You
Taylor Swift - You Belong With Me
Britney Spears - Toxic
Limp Bizkit - Behind Blue Eyes
Evanescence - Bring Me To Life
Nickelback - How You Remind Me
Kelly Clarkson - Breakaway
RBD - Rebelde
Pitty - Na Sua Estante
Christina Aguilera - Beautiful
Simple Plan - Welcome To My Life
Avril Lavigne - My Happy Ending
Linkin Park - Numb
Justin Timberlake - Cry Me a River
Maroon 5 - This Love
Akon - Lonely
Destiny's Child - Survivor
The Pussycat Dolls - Don't Cha (feat. Busta Rhymes)
50 Cent - Candy Shop (feat. Olivia)
blink-182 - I Miss You
CPM 22 - Um Minuto Para o Fim do Mundo
Green Day - Boulevard of Broke

In [39]:
songsDF.shape

(100, 2)

In [40]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

for index,row in songsDF.iterrows():
    song = row['Song']

    #Se obtiene el link del cual se debe tomar la información
    driver.get('https://www.letras.com')

    #Localizar la barra de busqueda
    campo = driver.find_element(By.ID,'main_suggest')

    #Usar el driver para encontrar la canción
    campo.send_keys(f'{song} original lyrics')
    campo.send_keys(Keys.ENTER)
    try:
        #Esperar a que cargue el contenido de la segunda pagina
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.gs-title')))

        #Clickar el primer resultado
        driver.find_element(By.CSS_SELECTOR, 'a.gs-title').click()
 
        #Esperar el contenido de la tercera pagina
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.lyric-original p')))

        #Encontrar todas las divisones de las letras
        letras = driver.find_elements(By.CSS_SELECTOR, 'div.lyric-original p')

        #Juntar la letra de las caciones
        letra_completa = "\n".join([p.text for p in letras])

        #Agregar la letra de la canción al dataframe
        songsDF.at[index,'Letra'] = letra_completa
    except Exception as e:
        #Manejar las excepciones
        songsDF.at[index, 'Letra'] = "Not Found"
        print(f"No se pudo obtener la letra de {song} por: {str(e)}")



driver.quit()

No se pudo obtener la letra de Mana y Juan Luis Guerra . Bendita tu luz por: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7817DB125+29573]
	(No symbol) [0x00007FF78174FF50]
	(No symbol) [0x00007FF78160B6EA]
	(No symbol) [0x00007FF78165F815]
	(No symbol) [0x00007FF78165FA6C]
	(No symbol) [0x00007FF7816AB917]
	(No symbol) [0x00007FF78168733F]
	(No symbol) [0x00007FF7816A86BC]
	(No symbol) [0x00007FF7816870A3]
	(No symbol) [0x00007FF7816512DF]
	(No symbol) [0x00007FF781652441]
	GetHandleVerifier [0x00007FF781B0C76D+3377613]
	GetHandleVerifier [0x00007FF781B57B67+3685831]
	GetHandleVerifier [0x00007FF781B4CF8B+3641835]
	GetHandleVerifier [0x00007FF78189B2A6+816390]
	(No symbol) [0x00007FF78175B25F]
	(No symbol) [0x00007FF781757084]
	(No symbol) [0x00007FF781757220]
	(No symbol) [0x00007FF78174607F]
	BaseThreadInitThunk [0x00007FFDA48C257D+29]
	RtlUserThreadStart [0x00007FFDA4FEAF28+40]

No se pudo obtener la letra de La Prohibida . Labios de Hiel por: Message: 
Stacktrace:
	GetHandle

In [11]:
##songsDF.to_excel('SongsDF.xlsx')

In [41]:
songsDF.to_excel('SongsDFEspañol.xlsx')